
# Libs

In [6]:
import torch
from PIL import Image
import numpy as np
from torch.utils.data import Dataset


import numpy as np
from torch.utils.data import ConcatDataset, DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10, CIFAR100
import os
import torch

from torch import optim
from collections import OrderedDict

import torch
import torch.nn as nn


# Helper

In [7]:
class BaseDataset(Dataset):
    def __init__(self, data, targets, transform, target_transform=None):
        self.targets = targets
        self.data = data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = Image.fromarray(self.data[index])
        if self.transform is not None:
            x = self.transform(x)

        y = self.targets[index]
        if self.target_transform is not None:
            y = self.target_transform(y)

        return x, y


def sparse_to_coarse(targets):
    coarse_labels = np.array([4, 1, 14, 8, 0, 6, 7, 7, 18, 3,
                              3, 14, 9, 18, 7, 11, 3, 9, 7, 11,
                              6, 11, 5, 10, 7, 6, 13, 15, 3, 15,
                              0, 11, 1, 10, 12, 14, 16, 9, 11, 5,
                              5, 19, 8, 8, 15, 13, 14, 17, 18, 10,
                              16, 4, 17, 4, 2, 0, 17, 4, 18, 17,
                              10, 3, 2, 12, 12, 16, 12, 1, 9, 19,
                              2, 10, 0, 1, 16, 12, 9, 13, 15, 13,
                              16, 19, 2, 4, 6, 19, 5, 5, 8, 19,
                              18, 1, 2, 15, 6, 0, 17, 8, 14, 13])
    return coarse_labels[targets]

# Train

In [8]:
def get_train_transforms():
    transform_train = transforms.Compose([transforms.Resize(32),
                                          transforms.RandomCrop(32),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                               std=[0.5, 0.5, 0.5])
    ])
    return transform_train


def get_nomral_dataset_train(dataset_name, label, data_path, download):
    if dataset_name == 'cifar10':
        normal_train_ds = CIFAR10(data_path, train=True, download=download)
    elif dataset_name == 'cifar100':
        normal_train_ds = CIFAR100(data_path, train=True, download=download)
        normal_train_ds.targets = sparse_to_coarse(normal_train_ds.targets)
    else:
        raise NotImplementedError()

    normal_data = normal_train_ds.data[np.array(normal_train_ds.targets) == label]
    normal_train_ds = BaseDataset(normal_data, [0] * len(normal_data), get_train_transforms())
    return normal_train_ds

In [9]:
# TEST
normal_train_ds = get_nomral_dataset_train('cifar10', 0, './CIFAR10', True)

normal_train_loader = DataLoader(normal_train_ds, batch_size=100, shuffle=True)

Files already downloaded and verified


# Test

In [10]:
def get_test_transforms():
    transform_test = transforms.Compose([transforms.Resize(32),
                                         transforms.CenterCrop(32),
                                         transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                             std=[0.5, 0.5, 0.5])])
    return transform_test



def get_dataset_test(normal_label, ano_label, data_path, download):
    test_cifar10 = CIFAR10(data_path, train=False, download=download)
    normal_test_data = test_cifar10.data[np.array(test_cifar10.targets) == normal_label]
    normal_test_ds = BaseDataset(normal_test_data, [0] * len(normal_test_data), get_test_transforms())

    test_cifar10 = CIFAR10(data_path, train=False, download=download)
    ano_test_data = test_cifar10.data[np.array(test_cifar10.targets) == ano_label]
    ano_test_ds = BaseDataset(ano_test_data, [1] * len(ano_test_data), get_test_transforms())


    return normal_test_ds, ano_test_ds

In [11]:
# TEST
normal_test_ds, ano_test_ds = get_dataset_test(0, 1, './CIFAR10', True)

normal_test_loader = DataLoader(normal_test_ds, batch_size=100, shuffle=True)
ano_test_loader = DataLoader(ano_test_ds, batch_size=100, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


# TEST Lib rary

In [12]:
from customdataset import get_nomral_dataset_train, get_dataset_test

In [14]:
normal_test_ds, ano_test_ds = get_dataset_test(0, 1, './CIFAR10', True)
normal_train_ds = get_nomral_dataset_train('cifar10', 0, './CIFAR10', True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# TSNE

In [15]:
from sklearn.manifold import TSNE

def get_2d_tsne(x):
    tsne = TSNE(n_components=2, random_state=0,  learning_rate='auto', init='random', perplexity=3)
    x_2d = tsne.fit_transform(x)

    return x_2d

# UNet

In [ ]:
class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=3, init_features=32):
        super(UNet, self).__init__()

        img_size = 32
        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        dim = 16*features * (img_size**2)/(2**8)
        self.fc1 = nn.Sequential(
            nn.Linear(int(dim/1), int(dim/2)),
            nn.BatchNorm1d(int(dim/2)),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(int(dim/2), int(dim/4)),
            nn.BatchNorm1d(int(dim/4)),
            nn.ReLU()
        )

        self.fc3 = nn.Sequential(
            nn.Linear(int(dim/4), int(dim/2)),
            nn.BatchNorm1d(int(dim/2)),
            nn.ReLU()
        )

        self.fc4 = nn.Sequential(
            nn.Linear(int(dim/2), int(dim/1)),
            nn.BatchNorm1d(int(dim/1)),
            nn.ReLU()
        )


        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x, y=None):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))
        bottleneck = self.bottleneck(self.pool4(enc4))

        fc1 = self.fc1(bottleneck.flatten(1))
        fc2 = self.fc2(fc1)

        self.compress_features = fc2

        if y is not None:
          fc3 = self.fc3(y)
        else:
          fc3 = self.fc3(self.compress_features)


        fc4 = self.fc4(fc3)

        bottleneck = fc4.view(bottleneck.shape)
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.tanh(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [ ]:
from torch import optim
from collections import OrderedDict

import torch
import torch.nn as nn


In [ ]:
normal_train_ds = get_nomral_dataset_train('cifar10', 0, './CIFAR10', False)

normal_train_loader = DataLoader(normal_train_ds, batch_size=100, shuffle=True)

In [ ]:
device = 'cuda'
model = UNet().to(device)
optimizer = optim.Adam(model.parameters())
critic = nn.MSELoss()

In [ ]:
normal_test_ds, ano_test_ds = get_dataset_test(0, 1, './CIFAR10', False)

normal_test_loader = DataLoader(normal_test_ds, batch_size=normal_test_ds.__len__(), shuffle=True)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
for x, _ in normal_test_loader:
    x_test = x
    x_test = x_test.to(device)

In [ ]:
loss_min = np.Inf
model.train()
for epoch in range(1000):
    torch.cuda.empty_cache()
    for x, _ in normal_train_loader:
        torch.cuda.empty_cache()
        x = x.to(device)
        optimizer.zero_grad()
        x_hat = model(x)
        loss = critic(x_hat, x)
        loss.backward()
        optimizer.step()

    x_hat = model(x_test)
    loss = critic(x_hat, x_test)
    print(f"Epoch{epoch} : Loss = {loss}")

# TSNE

In [ ]:
x_hat = model(x_test)

In [ ]:
x1 = model.compress_features.flatten(1)
x2 = x_test.flatten(1)

In [ ]:
x1 = x1.detach().cpu()
x2 = x2.detach().cpu()

In [ ]:
def get_2d_tsne(x):
    tsne = TSNE(n_components=2, random_state=0)
    x_2d = tsne.fit_transform(x)

    return x_2d

In [ ]:
plt.figure()
plt.scatter(get_2d_tsne(x1)[:,0], get_2d_tsne(x1)[:,1], label='Compress')
plt.scatter(get_2d_tsne(x2)[:,0], get_2d_tsne(x2)[:,1], label='Raw')
plt.legend()

In [ ]:
plt.figure()
plt.scatter(get_2d_tsne(x1)[:,0], get_2d_tsne(x1)[:,1], label='Compress')

# new

In [ ]:
normal_test_ds, ano_test_ds = get_dataset_test(0, 2, './CIFAR10', True)

normal_test_loader = DataLoader(normal_test_ds, batch_size=1000, shuffle=True)
ano_test_loader = DataLoader(ano_test_ds, batch_size=1000, shuffle=True)

In [ ]:
for x,_ in ano_test_loader:
  x_ano_1=x
  break

In [ ]:
for x,_ in normal_test_loader:
  x_normal=x
  break

In [ ]:
model = model.cpu()
model.eval()

y_normal = model(x_normal)
x_normal_compress = model.compress_features.flatten(1).detach().cpu()

y_ano_1 = model(x_ano_1)
x_ano_1_compress = model.compress_features.flatten(1).detach().cpu()

In [ ]:
x_normal = x_normal.flatten(1)
x_ano_1 = x_ano_1.flatten(1)

In [ ]:
plt.figure()
plt.scatter(get_2d_tsne(x_normal_compress)[:,0], get_2d_tsne(x_normal_compress)[:,1], label='Normal')
plt.scatter(get_2d_tsne(x_ano_1_compress)[:,0], get_2d_tsne(x_ano_1_compress)[:,1], label='Ano')
plt.legend()

# Add noise

In [ ]:
for x,_ in normal_test_loader:
  x_normal=x
  break

In [ ]:
model = model.cpu()
model.eval()

y_normal = model(x_normal)
x_normal_compress = model.compress_features.flatten(1).detach().cpu()

x_normal_compress_noise = torch.empty(512).normal_(0,5) + x_normal_compress
y_normal_compress_noise = model(x_normal, x_normal_compress_noise)

In [ ]:
x_normal_compress

In [ ]:
x_normal_compress_noise

In [ ]:
plt.figure()
plt.scatter(get_2d_tsne(x_normal_compress)[:,0], get_2d_tsne(x_normal_compress)[:,1], label='Normal')
plt.scatter(get_2d_tsne(x_normal_compress_noise)[:,0], get_2d_tsne(x_normal_compress_noise)[:,1], label='Ano')
plt.legend()

In [ ]:
y_normal_compress_noise.shape

In [ ]:
y_normal_compress_noise

In [ ]:
y_normal

In [ ]:
plt.imshow(y_normal_compress_noise[1].permute(1,2,0).detach().cpu())

In [ ]:
plt.imshow(y_normal[1].permute(1,2,0).detach().cpu())